In [1]:
import os, sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("."))
from src import get_soy_condition_features, SEVEN_STATES

YEAR_FROM = 1987
YEAR_TO   = 2024
STATES    = SEVEN_STATES
STATES_FILE = "data/processed/waob_features_states.csv" 

TARGET_YEAR_FORECAST = 2020

BASELINE_FORM_NO_D = "yield_bu_acre ~ trend + jun_shortfall + temp_JA + prec_JA + I(prec_JA**2)"
BASELINE_FORM_WITH_D = BASELINE_FORM_NO_D + " + dummy_2003"
AUGMENTED_FORM_NO_D = BASELINE_FORM_NO_D + " + gex_JA_min"
AUGMENTED_FORM_WITH_D = BASELINE_FORM_WITH_D + " + gex_JA_min"

plt.rcParams['figure.figsize'] = (6,4)
%matplotlib inline


## 1) Load datas

In [ ]:
_, cond_annual = get_soy_condition_features(YEAR_FROM, YEAR_TO, STATES)
states = pd.read_csv(STATES_FILE)

cols_keep = ["state","year","yield_bu_acre","harvest_ha","acres_harvested",
             "trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","dummy_2003"]
states = states[cols_keep].drop_duplicates(subset=["state","year"]).copy()

def normalize_keys(df):
    out = df.copy()
    out["state"] = out["state"].astype(str).str.strip().str.upper()
    out["year"] = pd.to_numeric(out["year"], errors="coerce").astype("Int64")
    return out

states = normalize_keys(states)
cond_annual = normalize_keys(cond_annual)

cond_annual = cond_annual.drop_duplicates(subset=["state","year"]).copy()

from src import SEVEN_STATES
states = states[states["state"].isin(SEVEN_STATES)].copy()

df = states.merge(cond_annual, on=["state","year"], how="left")

def add_us_weighted_row(df_in: pd.DataFrame) -> pd.DataFrame:
    if "harvest_ha" not in df_in.columns:
        raise ValueError("harvest_ha is required to compute US weights.")

    num_cols = df_in.select_dtypes(include=[np.number]).columns.tolist()
    num_cols = [c for c in num_cols if c not in ["harvest_ha", "year"]]

    def wmean(g: pd.DataFrame) -> pd.Series:
        w = g["harvest_ha"].fillna(0.0).astype(float)
        out = {c: (np.average(g[c].astype(float), weights=w) if w.sum() > 0 else np.nan)
               for c in num_cols}
        out["harvest_ha"] = float(w.sum())
        out["acres_harvested"] = float(g.get("acres_harvested", 0).fillna(0).sum())
        return pd.Series(out)
    
    us = (df_in.groupby("year", as_index=True)
               .apply(wmean, include_groups=False)
               .reset_index())

    us["state"] = "US"
    return pd.concat([df_in, us], ignore_index=True, sort=False)


df_full = add_us_weighted_row(df)
df_us   = df_full[df_full["state"]=="US"].sort_values("year").copy()

print("df_full shape:", df_full.shape, "| US years:", df_us["year"].min(), "→", df_us["year"].max())
display(df_us.tail(3))


df_full shape: (296, 18) | US years: 1988 → 2024


,state,year,yield_bu_acre,harvest_ha,acres_harvested,trend,jun_shortfall,temp_JA,prec_JA,prec_JA_sq,dummy_2003,gex_JA_mean,gex_JA_min,fair_JA_mean,pvp_JA_max,cond_index_JA_mean,gex_week31,gex_trend
293,US,2022,56.071932,1.233765e+08,304870000.0,35.0,0.451474,73.774935,3.838609,15.441285,0.0,30.234508,27.125365,28.525685,7.407190,2.422522,30.682373,-4.124315
294,US,2023,55.659603,1.174883e+08,290320000.0,36.0,1.023105,72.745083,3.851704,15.866829,0.0,27.149044,22.235757,30.814944,10.431610,2.356335,26.751636,1.551977
295,US,2024,57.812707,1.210496e+08,299120000.0,37.0,0.000000,72.674866,3.848636,15.281408,0.0,34.961023,32.231813,22.302825,5.429794,2.456012,35.542809,-0.962306


## 2) Utilities

In [3]:
def fit_ols(formula: str, data: pd.DataFrame):
    model = smf.ols(formula=formula, data=data).fit()
    return model

def metrics_from_model(m):
    rmse = float(np.sqrt(m.mse_resid)) if hasattr(m, "mse_resid") else np.nan
    return {
        "n_obs": int(m.nobs),
        "r2": float(m.rsquared),
        "r2_adj": float(m.rsquared_adj),
        "rmse": rmse
    }

def train_and_predict(formula: str, df_train: pd.DataFrame, df_predict: pd.DataFrame, pred_label: str):
    m = fit_ols(formula, df_train)
    yhat = m.predict(df_predict)
    out = df_predict[["state","year"]].copy()
    out[pred_label] = np.asarray(yhat, dtype=float)
    return m, out

def ensure_year(df_in: pd.DataFrame, state: str, year: int):
    sub = df_in[(df_in["state"]==state) & (df_in["year"]==year)].copy()
    if sub.empty:
        raise ValueError(f"No row for {state} {year} in data.")
    if len(sub) > 1:
        # Agrège les colonnes numériques par la moyenne, garde le premier pour le reste
        num_cols = sub.select_dtypes(include=[np.number]).columns.tolist()
        sub = (sub
               .groupby(["state","year"], as_index=False)
               .agg({**{c: "mean" for c in num_cols},
                     **{c: "first" for c in sub.columns if c not in num_cols and c not in ["state","year"]}}))
    return sub

## 3) National model tests

In [ ]:
# ----------------------------- Modele no Dummy (1988-2013) -------------
train_A = df_us[(df_us["year"] >= 1988) & (df_us["year"] <= 2013)].dropna(subset=["yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq"])
row_2020 = ensure_year(df_us, "US", 2020).dropna(subset=["trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq"])

mA, predA = train_and_predict(BASELINE_FORM_NO_D, train_A, row_2020, "forecast_2020_A")
metA = metrics_from_model(mA)

print("=== National A: train ≤2013, no dummy ===")
print(mA.summary())

def scalar(series, name="value", reduce="mean"):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty:
        raise ValueError(f"{name}: value is NaN/empty.")
    if len(s) > 1:
        return float(s.mean()) if reduce=="mean" else float(s.iloc[0])
    return float(s.iloc[0])

# ----------------------------- Modele with Dummy (1988-2019) -------------
need_cols = ["yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","dummy_2003"]
train_B = df_us[(df_us["year"] >= 1988) & (df_us["year"] <= 2019)].dropna(subset=need_cols)
mB, predB = train_and_predict(BASELINE_FORM_WITH_D, train_B, row_2020, "forecast_2020_B")
metB = metrics_from_model(mB)

print("\n=== National B: train ≤2019, with dummy_2003 ===")
print(mB.summary())


# ----------------------------- Forecast 2020 -------------
coefs = mB.params
wasde_2020   = 49.8
mean_prec = train_B["prec_JA"].mean()

overrides = {
    "Intercept": 1.0,                         
    "trend": float(train_B["trend"].max()),   
    "jun_shortfall": 0.0,                     
    "dummy_2003": 0.0,                        
    "I(prec_JA ** 2)": float(mean_prec**2),   
}

def sample_value(var: str) -> float:
    if var in overrides:
        return overrides[var]
    if var in train_B.columns:
        return float(train_B[var].mean())
    return np.nan

means = pd.Series({var: sample_value(var) for var in coefs.index})

waob_table = pd.DataFrame({
    "Coefficient": coefs,
    "Sample Average": means,
})
waob_table["Product"] = waob_table["Coefficient"] * waob_table["Sample Average"]

fcst_2020_B = float(waob_table["Product"].sum())

print(waob_table)
print(f"Actual 2020 = {wasde_2020:.2f}  |  Forecast = {fcst_2020_B:.2f}  |  Error = {fcst_2020_B - wasde_2020:.2f}")


=== National A: train ≤2013, no dummy ===
                            OLS Regression Results                            
Dep. Variable:          yield_bu_acre   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     20.54
Date:                Thu, 09 Oct 2025   Prob (F-statistic):           2.96e-07
Time:                        20:29:06   Log-Likelihood:                -53.157
No. Observations:                  26   AIC:                             118.3
Df Residuals:                      20   BIC:                             125.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------


## 4) State-level models (≤2019) and predict 2020

In [6]:
rows = []
coefs_rows = []

for st in STATES:
    d = df_full[df_full['state']==st].copy()
    train = d[(d["year"]>=1988)&(d["year"]<=2019)].dropna(subset=["yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","dummy_2003"])
    try:
        row20 = ensure_year(d, st, TARGET_YEAR_FORECAST).dropna(subset=["trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq"])
    except ValueError:
        continue

    if train.empty or row20.empty:
        continue

    m, pred = train_and_predict(BASELINE_FORM_WITH_D, train, row20, "forecast_2020")
    met = metrics_from_model(m)
    y_true = (row20["yield_bu_acre"].item()
            if not row20["yield_bu_acre"].isna().all() else np.nan)
    y_hat = pred["forecast_2020"].item()

    rows.append({
        "state": st,
        "n_obs": met["n_obs"],
        "r2": met["r2"],
        "y2020_actual": y_true,
        "y2020_forecast": y_hat,
        "y2020_error": y_hat - y_true
    })

    prms = m.params.to_dict()
    prms["state"] = st
    coefs_rows.append(prms)

state_results_baseline = pd.DataFrame(rows).sort_values("state")
state_coefs_baseline = pd.DataFrame(coefs_rows).set_index("state").sort_index()

print("=== State baseline results (≤2019 + dummy → 2020) ===")
display(state_results_baseline)
print("\n=== State baseline coefficients ===")
display(state_coefs_baseline)


=== State baseline results (≤2019 + dummy → 2020) ===


,state,n_obs,r2,y2020_actual,y2020_forecast,y2020_error
0,IA,32,0.867209,55.2,51.208345,-3.991655
1,IL,32,0.822326,60.8,55.363515,-5.436485
2,IN,32,0.816186,59.6,54.128435,-5.471565
5,MN,32,0.683502,51.2,49.555970,-1.644030
4,MO,32,0.848950,50.8,45.660581,-5.139419
6,NE,32,0.946737,59.6,58.123855,-1.476145
3,OH,32,0.837350,55.8,51.183739,-4.616261



=== State baseline coefficients ===


,Intercept,trend,jun_shortfall,temp_JA,prec_JA,I(prec_JA ** 2),dummy_2003
state,,,,,,,
IA,35.881241,0.513357,-1.348488,-0.172079,6.617987,-0.627333,-7.203319
IL,51.718901,0.568858,-1.236954,-0.318052,2.459912,-0.035872,-6.530733
IN,22.667998,0.493964,-1.748350,0.051162,3.490173,-0.154720,-9.066123
MN,-24.780976,0.406495,-2.862298,0.469926,12.949095,-1.479647,-1.338426
MO,60.062763,0.498643,-1.060150,-0.559174,4.083478,-0.302145,-4.354370
NE,34.788512,0.799946,-1.105230,-0.302229,11.326726,-1.421015,2.753716
OH,6.425673,0.501372,-1.429846,-0.016470,13.258279,-1.364629,-3.640370


## 5) National model adding crops conditions

In [ ]:
# === 5) National model with `gex_JA_min` ===

need_aug = ["yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","gex_JA_min"]

pool_aug = df_us.dropna(subset=need_aug).copy()
if pool_aug.empty:
    raise ValueError("No US rows with non-missing gex_JA_min + WAOB features. Check your condition features merge.")

first_year_aug = int(pool_aug["year"].min())

need_cols_aug = need_aug + ["dummy_2003"]
train_Bp = df_us[(df_us["year"] >= first_year_aug) & (df_us["year"] <= 2019)].dropna(subset=need_cols_aug)

if len(train_Bp) < 5:
    raise ValueError(f"B' training too short ({len(train_Bp)} rows).")

mBp = smf.ols(formula=AUGMENTED_FORM_WITH_D, data=train_Bp).fit()
metBp = {"n_obs": int(mBp.nobs), "r2": float(mBp.rsquared), "r2_adj": float(mBp.rsquared_adj),
         "rmse": float(np.sqrt(mBp.mse_resid))}

print(f"\\n=== National': start={first_year_aug} to 2019 (+ dummy + gex_JA_min) ===")
print(mBp.summary())



\n=== National': start=1988 to 2019 (+ dummy + gex_JA_min) ===
                            OLS Regression Results                            
Dep. Variable:          yield_bu_acre   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     63.53
Date:                Thu, 09 Oct 2025   Prob (F-statistic):           6.06e-14
Time:                        20:30:22   Log-Likelihood:                -57.637
No. Observations:                  32   AIC:                             131.3
Df Residuals:                      24   BIC:                             143.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

## 6) State-level models

In [8]:
rows_aug = []
coefs_rows_aug = []

for st in STATES:
    d = df_full[df_full['state']==st].copy()
    need = ["yield_bu_acre","trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","dummy_2003","gex_JA_min"]
    train = d[(d["year"]>=1988)&(d["year"]<=2019)].dropna(subset=need)
    try:
        row20 = ensure_year(d, st, TARGET_YEAR_FORECAST).dropna(subset=["trend","jun_shortfall","temp_JA","prec_JA","prec_JA_sq","gex_JA_min"])
    except ValueError:
        continue

    if train.empty or row20.empty:
        continue

    m, pred = train_and_predict(AUGMENTED_FORM_WITH_D, train, row20, "forecast_2020_aug")
    met = metrics_from_model(m)
    y_true = (
        float(row20["yield_bu_acre"].iloc[0])
        if not row20["yield_bu_acre"].isna().all()
        else np.nan
    )
    y_hat = float(pred["forecast_2020_aug"].iloc[0])
    
    rows_aug.append({
        "state": st,
        "n_obs": met["n_obs"],
        "r2": met["r2"],
        "r2_adj": met["r2_adj"],
        "rmse": met["rmse"],
        "y2020_actual": y_true,
        "y2020_forecast": y_hat,
        "y2020_error": y_hat - y_true
    })

    prms = m.params.to_dict()
    prms["state"] = st
    coefs_rows_aug.append(prms)

state_results_aug = pd.DataFrame(rows_aug).sort_values("state")
state_coefs_aug = pd.DataFrame(coefs_rows_aug).set_index("state").sort_index()

print("=== State augmented results (≤2019 + dummy + gex_JA_min → 2020) ===")
display(state_results_aug)
print("\n=== State augmented coefficients ===")
display(state_coefs_aug)


=== State augmented results (≤2019 + dummy + gex_JA_min → 2020) ===


,state,n_obs,r2,r2_adj,rmse,y2020_actual,y2020_forecast,y2020_error
0,IA,32,0.927830,0.906781,2.066174,55.2,52.151253,-3.048747
1,IL,32,0.869674,0.831662,3.076858,60.8,56.512544,-4.287456
2,IN,32,0.882508,0.848239,2.494583,59.6,55.423514,-4.176486
5,MN,32,0.864602,0.825111,2.358543,51.2,50.008239,-1.191761
4,MO,32,0.911756,0.886019,2.154208,50.8,47.730487,-3.069513
6,NE,32,0.965196,0.955045,1.816031,59.6,58.350371,-1.249629
3,OH,32,0.929810,0.909338,1.973676,55.8,51.871041,-3.928959



=== State augmented coefficients ===


,Intercept,trend,jun_shortfall,temp_JA,prec_JA,I(prec_JA ** 2),dummy_2003,gex_JA_min
state,,,,,,,,
IA,12.117334,0.441644,0.353147,0.192967,0.560207,-0.042330,-3.601640,0.411007
IL,21.664285,0.555100,-0.225743,0.054141,0.929050,-0.018124,-7.255325,0.312870
IN,6.281875,0.487067,-0.125769,0.250424,1.467275,-0.033435,-7.024709,0.306879
MN,-3.979479,0.220819,-1.654899,0.266348,6.099783,-0.687963,1.046526,0.366567
MO,33.556856,0.427372,-0.175535,-0.223385,2.737411,-0.182925,-2.247719,0.278384
NE,7.800592,0.694254,-0.317246,0.105514,7.540715,-0.978581,2.013834,0.211747
OH,-5.313241,0.508772,-0.317202,0.228720,6.840843,-0.709729,-3.200695,0.342486


## 7) Save outputs

In [9]:
OUT_DIR = "data/results"
os.makedirs(OUT_DIR, exist_ok=True)

frames = [
    pd.Series(metA, name="value").reset_index().assign(source="national_A"),
    pd.Series(metB, name="value").reset_index().assign(source="national_B"),
    pd.Series(metBp, name="value").reset_index().assign(source="national_Bp"),
    state_results_baseline.assign(source="state_results_baseline"),
    state_coefs_baseline.assign(source="state_coefs_baseline"),
    state_results_aug.assign(source="state_results_aug"),
    state_coefs_aug.assign(source="state_coefs_aug"),
]

final_df = pd.concat(frames, ignore_index=True)
final_df.to_csv(f"{OUT_DIR}/waob_outputs_all.csv", index=False)
